In [1]:
import json
import time
import latexcodec
import codecs
import glob
import os 
import latexcodec

def save_dict2json(json_path, dict_md5):
    with open(json_path, 'w') as fp:
        json.dump(dict_md5, fp)

def load_json2dict(json_path):
    if os.path.exists(json_path):
        json_file = open(json_path)
        json_data = json.load(json_file)
    else:
        json_data = None
    return json_data

def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r ran for  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

name_mapping = {
    "Joeran S.": "J.S.",
}

def authors_to_string(names, bib_key):
    string_authors = ''
    d = ', '
    for idx, name in enumerate(names):
        first, von, last, jr = name
        if first:
            if first in name_mapping:
                first = name_mapping[first]
            else:
                first = first[0] + '.'
        if idx == len(names)-2:
            d = ' and '
        if idx == len(names)-1:
            d = ''
        string_authors += ' '.join(part for part in [first, von, last, jr] if part) + d
    return string_authors

# pasre bib file
def get_blocks(content, start_character='@', delim=('{','}')):
    '''
    returns all blocks (entries enclosed by the specified delimiters)
    start_character will look backwards from the start of the block for this character
    the result will be a tuple of two strings: from start character to start of the block, and the block content
    '''
    blocks = []
    
    delim_start, delim_end = delim
    delimiter_stack = []
    for i, c in enumerate(content):
        if c == '{':
            delimiter_stack.append(i)
        elif c == '}' and delimiter_stack:
            start = delimiter_stack.pop()
            if len(delimiter_stack)==0:
                start_index = content.rfind(start_character, 0, start)
                blocks.append((content[start_index: start], content[start + 1: i]))
    return blocks

assert [x for x in get_blocks('abc = {test}, bac = {test2}', 'a')] == [('abc = ', 'test'), ('ac = ', 'test2')]


def decode_name(name):
    parsed_name = []
    for name_part in name:
        name_part = name_part.strip()
        if '\\' in name_part:
            name_part = codecs.decode(name_part, "ulatex")
        parsed_name.append(name_part)
    return parsed_name


def parse_name(name):
    '''
    assumes this format:
    https://tex.stackexchange.com/questions/557/how-should-i-type-author-names-in-a-bib-file
    returns a tuple (first, von, last, jr)
    '''
    parts = name.strip(',').split(',')
    
    # "First von Last"
    if len(parts)==1:
        s, e = (name.index(' '), name.rfind(' ')) if ' ' in name else (0, 0)
        first = name[:s]
        von = name[s:e]
        last = name[e:]
        jr = ''
        
    # "von Last, First"
    elif len(parts)==2: 
        first = parts[1]
        e = parts[0].rfind(' ') if ' ' in parts[0] else 0
        von = parts[0][:e]
        last = parts[0][e:]
        jr = ''
        
    # "von Last, Jr, First"
    elif len(parts)==3: 
        first = parts[2]
        e = parts[0].rfind(' ') if ' ' in parts[0] else 0
        von = parts[0][:e]
        last = parts[0][e:]
        jr = parts[1]
        
    else:
        print('warning! bibtex format error in name "{}"'.format(''.join(name)))
        first, von, last, jr = '', '', name, ''  
    
    nfirst = ''
    for f in first.strip().split('.'):
        f = f.strip()
        f = f.capitalize()
        if len(f) == 1:
            f += '.'    
        nfirst = ' '.join([nfirst, f])
    
    #post process von to second names
    nvon = ''
    for v in von.strip().split():
        v = v.strip()
        if v[0].isupper():
            nfirst = ' '.join([nfirst, v])
        else:
            nvon = ' '.join([nvon, v])
    
    #post process first and second names
    nfirst2 = ''
    for f in nfirst.strip().split():
        f = f.strip()
        f = f.capitalize()
        if len(f) == 1:
            f += '.'    
        nfirst2 = ' '.join([nfirst2, f]) 
        
    last = '-'.join(last.strip().split())
        
    return decode_name((nfirst2, nvon, last, jr))


assert parse_name('Bart Liefers') == ['Bart', '', 'Liefers', '']
assert parse_name('Bart von Liefers') == ['Bart', 'von', 'Liefers', '']
assert parse_name('Liefers, Bart') == ['Bart', '', 'Liefers', '']
assert parse_name('von Liefers, Bart') == ['Bart', 'von', 'Liefers', '']
# assert parse_name('von Liefers, Jr, Bart') == ['Bart', 'von', 'Liefers', 'Jr']


def parse_block_content(bib_item_text):
    bib_item = {}
    
    # split lines and remove ',' at the end
    bib_item_text = bib_item_text.replace('\r', '\n').replace('\r\n', '\n')
    lines  = bib_item_text.split(',\n')
    
    # bib key
    bib_key = lines[0].lower()
    
    # parse lines
    for line in lines[1:]:
        
        # check if line is parseable
        if '=' not in line or line == '':
            continue 
            
        # split by tags notated by '='   
        key, value = line.split('=', 1)

        # strip unneccary symbols
        key = key.lower().strip()
        value = value.strip().strip('{').strip('}')
                    
        # set bib_item
        bib_item[key] = value
    
    return bib_key, bib_item
 
def single_author(author_string):
    splits = author_string.split(',')
    splits = [s.strip() for s in splits]
    return len(splits) == 2 and not(' ' in splits[0] and ' ' in splits[1])

def split_authors(author_string):
    authors = []
    if 'and' in author_string.lower() or single_author(author_string):
        authors = author_string.replace('AND', 'and').split(' and ')
    else:
        authors = author_string.split(',')
    authors = [a.strip().replace('{', '').replace('}', '') for a in authors]
    return authors

def read_bibtex_file(filename):
    bib_items = {}
    string_rules = {}
    with open(filename, 'rb') as f:
        content = f.read().decode('utf-8-sig')

    for block in get_blocks(content):
        block_name, block_content = block
        if block_name == '@Comment':
            continue
        elif block_name == '@String':
            k, v = [x.strip() for x in block_content.split('=')]
            string_rules[k] = v  
        else: #bib item text

            # parse bib item text
            bib_key, bib_item = parse_block_content(block_content)
            
            # update type
            bib_item['type'] = block_name.strip('@').lower()
            
            # update journal name
            if 'journal' in bib_item:
                bib_item['journal'] = string_rules[bib_item['journal']].strip('_').replace('_', ' ') if bib_item['journal'] in string_rules else bib_item['journal']

            if 'author' in bib_item:
                bib_item['author'] = list(map(parse_name, split_authors(bib_item['author'])))
                bib_item['authors'] = authors_to_string(bib_item['author'], bib_key)

            if 'abstract' in bib_item:
                bib_item['abstract'] = bib_item['abstract'].replace('{', '').replace('}', '').replace('\\', '')
                
            if 'title' in bib_item:
                bib_item['title'] = bib_item['title'].replace('{', '').replace('}', '').replace('\\', '')
                                                                                                      
            if 'copromotor' in bib_item:
                try:
                    bib_item['author'] += list(map(parse_name, split_authors(bib_item['copromotor'])))
                except:
                    print('bib_key cp', bib_key)
           
            if 'promotor' in bib_item:
                try:
                    bib_item['author'] += list(map(parse_name, split_authors(bib_item['promotor'])))
                except:
                    print('bib_key p', bib_key)

            bib_items[bib_key.lower()] = bib_item
    return bib_items


# authors
def get_list_researchers(members_path):
    list_researchers = {}
    for people_md_path in glob.glob(members_path + '/*.md'):
        with open(people_md_path) as fp:
            # parse md file
            tags = {line.split(':')[0]:line.split(':')[1].strip().split() for line in (fp) if len(line.split(':')) > 1}

            # get publication name
            research_name = [n for n in tags['pub_name']] if 'pub_name' in tags else [n for n in tags['name']]
            name = '-'.join(tags['name'])
            groups =  [group.strip(',')  for group in tags['groups']]
        # append researches 
        list_researchers[name] = (research_name, groups)
    return list_researchers


# author publications
def get_publications_by_author(bib_items, list_researchers):
    """
    Get all publication per author
    returns dictionary with authorname as key and list of bib_keys as value
    """
    author_bibkeys = {}
    for bib_key, bib_item in bib_items.items():
        if "author" not in bib_item:
            continue
        authors = bib_item["author"]
        for name, value in list_researchers.items():
            researcher_name, _, _, _, _ = value
            firstname = researcher_name[0].lower()
            lastnames = [n.lower() for n in researcher_name[1:]]

            if len(lastnames) > 1:
                # This fixes issue #10 for lastnames connected with a dash (-)
                lastnames.append("-".join(lastnames))

            for author_pub in authors:
                if match_author_publication(firstname, lastnames, author_pub, bib_key):
                    author_bibkeys.setdefault(name.lower(), []).append(bib_key)
    return author_bibkeys


def match_author_publication(firstname, lastnames, author, bib_key):
    # This function selects authors with the same lastname and matches the first name.
    # For instance, 'A. Patel' will always represent 'Ajay Patel' and not 'Anup Patel'.
    # If the bib file contains 'A Patel' then, it will associate the bibentry to 'Ajay Patel'.
    # If the bib file contains 'M F L Meijs' then, this script will not associate the bibentry to 'Midas Meijs'
    #  because it will check for the existence of von and jr (F and L in the example)
    author = [xname.replace('.', ' ').strip() for xname in author]
    try:
        first, von, last, jr = author
    except:
        print(author)
    first = first.lower()
    last = last.lower()
    jr = jr.lower()
    
    # Additional variable that may help to avoid incorrect name matching #77
    von_last = '-'.join([von, last])
    von_last = von_last.replace(' ', '-').lower()

    if last.lower() in lastnames:
        # First match based on last name
        if len(first) > 1 and first.lower() == firstname.lower() or len(jr) > 1 and jr.lower() in lastnames:
            # Easy match, the first name is complete and matches up
            return True
        elif len(first) > 1 and ' ' in first:
            # Incomplete match, some bib entries have authors as 'R Manniesing' instead of the full name
            # or 'J A W M van der Laak' where firstname contains 'J A W M'
            # or 'Jeroen AWM van der Laak' where firstname contains 'Jeroen A W M'
            # This piece of code makes sure there is only one name and no spaces in between
#             von = ' '.join(first.split(' ')[1:]) + ' ' + von
            first = first.split(' ')[0].lower()
            if first == firstname.lower():
                return True
            # if 'first' contains a single letter, it will continue

        if len(first) == 1 and first[0].lower() == firstname[0].lower():
            # If only one letter is provided as first name (incomplete in the bib entry).
            # An additional variable stores the initial lastnames
            initials_lastnames = [x[0].lower() for x in lastnames]
            if (len(von) == 0 and len(jr) == 0):
                # If there is no 'von' neither 'jr', then it is a match
                return True
            elif (len(jr) >= 1 and jr[0].lower() in initials_lastnames):
                # If 'jr' contains something, it will have to be listed on 'initials_lastnames'
                # to become a match
                return True
            elif (len(von.strip()) >= 1 and von.strip()[0].lower() in initials_lastnames):
                # If 'von' contains something, it will have to be listed on 'initials_lastnames'
                # to become a match
                return True
            elif '-' != von_last[0] and len(lastnames) >= 2 and lastnames[-1] in von_last:
                # If none of the previous methods worked, an additional checkup is done.
                # This is done only when having at least two last names.
                # the last lastname should be in 'von_last'.
                # For instance 'J A W M van der Laak' will become 'A-W-M-van-der-Laak', this matches up with 'van-der-laak'
                return True
        return False
    else:
        return False
    

In [2]:
parse_name('Bart von Liefers')

['Bart', 'von', 'Liefers', '']

In [2]:
# get bib_items
@timeit
def parse_bib_file():
    print('parsing bib file...')
    bib_items = read_bibtex_file('../content/diag.bib')
    
    print('retreiving list of diag members')
    list_researchers = get_list_researchers(r"C:\Users\drepeeters\PycharmProjects\website-content\content\pages\members")  
    
    print('mapping bib keys to authors')
    author_bib_keys = get_publications_by_author(bib_items, list_researchers)
    
    # print('saving bibitems.json')
    # save_dict2json('/home/mart/bitems.json', bib_items)
    
    # print('saving auhtorbibkeys.json')
    # save_dict2json('/home/mart/authorkeys.json', author_bib_keys)
    
    print('creating author md files')
    print('creating publication md files')
    return list_researchers, bib_items, author_bib_keys

In [ ]:
list_researchers, bib_items, author_bib_keys = parse_bib_file()

    

In [11]:
researchers = get_list_researchers(r"C:\Users\drepeeters\PycharmProjects\website-content\content\pages\members")
researchers["Stan-Noordman"]

(['Stan', 'Constant', 'Richard', 'Noordman'], ['diag'])

In [21]:
get_publications_by_author()

'J.S. Bosma, A. Saha, M. Hosseinzadeh, I. Slootweg, M. de Rooij and H. Huisman'

In [11]:
bib_items['bult19a']['abstract']

'Grading prostate cancer is a time-consuming process and suffers from high inter- and intra-observer variability. Advances in computer-aided diagnosis have shown promise in improving histopathological diagnosis. We trained a deep learning system using data retrieved from the patients records to grade digitized prostate biopsies. Our system is the first that can automatically classify background, benign epithelium, Gleason 3, 4, and 5 on a gland-by-gland level in prostate biopsies. 532 glass slides containing 2162 prostate biopsies, evaluated by an experienced urogenital pathologist were collected and scanned. 596 biopsies were kept separate for evaluation, the remaining 1576 were used to train the deep learning algorithm (see table for Gleason grade distribution). A single label denoting the Gleason score (e.g. 3+4=7) was available for each biopsy, without information on tumor location or volume. To generate detailed annotations for training we used two previously trained deep learning

In [34]:
for bkey in author_bib_keys['John-Melle-Bokhorst']: 
    print(bib_items[bkey]['title'])
    print(bib_items[bkey]['authors'])
    print(bib_items[bkey].keys())

Assessment of tumor buds in colorectal cancer. A large-scale international digital observer study
J. Bokhorst, H. Dawson, A. Blank, I. Zlobec, A. Lugli, M. Vieth, R. Kirsch, M. Urbanowicz, S. Brockmoeller, J. Flejou, L. Rijstenberg, J. van der Laak, F. Ciompi and I. Nagtegaal
dict_keys(['author', 'title', 'booktitle', 'year', 'abstract', 'optnote', 'type', 'authors'])
Learning from sparsely annotated data for semantic segmentation in histopathology images
J. Bokhorst, H. Pinckaers, P. van Zwam, I. Nagetgaal, J. van der Laak and F. Ciompi
dict_keys(['author', 'title', 'booktitle', 'year', 'volume', 'pages', 'url', 'abstract', 'file', 'optnote', 'type', 'authors'])
Automatic Detection of Tumor Budding in Colorectal Carcinoma with Deep Learning
J. Bokhorst, L. Rijstenberg, D. Goudkade, I. Nagtegaal, J. van der Laak and F. Ciompi
dict_keys(['author', 'title', 'booktitle', 'year', 'date', 'publisher', 'month', 'doi', 'url', 'abstract', 'file', 'optnote', 'type', 'authors'])


In [37]:
bib_items['swid18']

{'author': [['Zaneta', '', 'Swiderska-Chadaj', ''],
  ['Hans', '', 'Pinckaers', ''],
  ['Mart', 'van', 'Rijthoven', ''],
  ['Maschenka', '', 'Balkenhol', ''],
  ['Margarita', '', 'Melnikova', ''],
  ['Oscar', '', 'Geessink', ''],
  ['Quirine', '', 'Manson', ''],
  ['Geert', '', 'Litjens', ''],
  ['Jeroen', 'van der', 'Laak', ''],
  ['Francesco', '', 'Ciompi', '']],
 'title': 'Convolutional Neural Networks for Lymphocyte detection in Immunohistochemically Stained Whole-Slide Images',
 'booktitle': 'MIDL',
 'year': '2018',
 'url': 'https://openreview.net/forum?id=rk0xLisiM',
 'abstract': 'Recent advances in cancer immunotherapy have boosted the interest in the role played by the immune system in cancer treatment. In particular, the presence of tumor-infiltrating lymphocytes (TILs) have become a central research topic in oncology and pathology. Consequently, a method to automatically detect and quantify immune cells is of great interest. In this paper, we present a comparison of different

In [6]:
import latexcodec
import codecs

In [7]:
# author_bib_keys['clara-i-sánchez']
codecs.decode(bib_items['abas05a']['authors'], "ulatex")

'D. Abásolo, C. Gómez, J. Poza, M. García, C. Sánchez and M. López'

In [8]:
author_bib_keys['zaneta-swiderska-chadaj']

['rijt18', 'swid18', 'swid19']

In [9]:
bib_items['tell19']['pages']

'101544'

In [29]:
def save_md_file(output_path, md_content):
    file = open(output_path, 'w', encoding='utf-8')
    file.write(md_content)
    file.close()
    
def create_author_md_files():
    for name, bib_keys in author_bib_keys.items():
        author_name = name.replace('_', ' ')
        groups = list_researchers[name][1]
        
        md_string = 'title: Publications of ' + author_name.replace('-', ' ') + '\n'
        md_string += 'template: publications-author\n'
        md_string += 'author: ' + name +'\n'
        md_string += 'author_name: ' + author_name.replace('-', ' ') + '\n'
        md_string += 'groups: ' + ','.join(groups) + '\n'
        md_string += 'bibkeys: ' + ','.join(bib_keys)
        md_file_name = name.lower() + '.md'
        
def create_publication_md():
    for bib_key, bib_item in bib_items.items():
        diag_authors = []
        groups = set()
        for name, bib_keys in author_bib_keys.items():
            for bkey in bib_keys:
                if bib_key == bkey:
                    diag_authors.append(name)
                    for group in list_researchers[name][1]:
                        groups.add(group)
                    md_string = 'title: ' + bib_item['title'] + '\n'
                    md_string += 'template: publication\n'
                    md_string += 'bibkey: ' + bib_key + '\n'
                    md_string += 'diag_authors: ' + ','.join(diag_authors) +'\n'
                    md_string += 'groups: ' + ','.join(groups) + '\n'
                    md_string += 'journal: NA \n' if 'journal' not in bib_item else 'journal: ' + bib_item['journal'] + '\n'
                    md_string += 'year: NA \n' if 'year' not in bib_item else 'year: ' + bib_item['year'] + '\n'
                    md_string += 'doi: NA \n' if 'doi' not in bib_item else 'doi: ' + bib_item['doi'] + '\n'
                    md_string += 'url: NA \n' if 'url' not in bib_item else 'url: ' + bib_item['url'] + '\n'
                    md_string += 'authors: ' + bib_item['authors'] + '\n'
                    md_string += 'has_pdf: True' + if 'file' in bib_item else 'has_pdf: False'
                    md_string += '' if 'abstract' not in bib_item else bib_item['abstract']
                    md_file_name = '/home/mart/Desktop/bib_files/' + bib_key + '.md'
                    save_md_file(md_file_name, md_string)
                    

In [231]:
create_author_md_files()

title: Publications of Rashindra Manniesing
template: publications-author
author: Rashindra-Manniesing
author_name: Rashindra Manniesing
groups: diag,neuro
bibkeys: deso19,avoi13,beck16,beck19a,boom14,boom12,boom12a,boom13,ciet11,ciet14,durm12,durm14,durm14a,eerd18,firo13,firo10a,firo10,firo12,firo12a,firo08,ghaf18,gish14,hame13,heuv16,koek12,koop08,koop08a,koop06,kort16,kort15,kosc18,kuij10,laue13,leem17,leem18c,leem18b,leem19b,leem18a,mann99a,mann06e,mann09a,mann09b,mann16a,mann00,mann99,mann04,mann98,mann10a,mann05a,mann07a,mann04a,mann06,mann06c,mann17,mann16,mann10,mann16c,mann04b,mann06a,mann08a,mann06b,mann07,mann09,mann09c,meij18c,meij18a,meij18,meij17a,meij18b,meij19a,meij17,meij16,mook09,mord13,mord12,oei13,oei14,oei14a,oei18,oei17,oei17a,oei12,pate17b,pate17a,pate19,pate16,pate18,pate17,pegg17a,roll09,scha07,stan18,tan16,leem19,leem19c,vuka12,vuka09a,vuka11,vuka08,vuka10,vuka09,wiel10,wiel11,pate19a


In [13]:
bib_items = load_json2dict('/home/mart/radboudumc/website-content/content/bibitems.json')
bib_author_keys = load_json2dict('/home/mart/radboudumc/website-content/content/authorkeys.json')

In [42]:
s = b"I{\v{s}}gum"

In [43]:
a = codecs.decode(s, 'ulatex')

TypeError: decoding with 'ulatex' codec failed (TypeError: cannot use a string pattern on a bytes-like object)

In [31]:
a

'I{\x0b{s}}gum'

In [29]:
bib_items['leem19b']

{'author': [['Sil C.', 'van de', 'Leemput', ''],
  ['Jonas', '', 'Teuwen', ''],
  ['Bram', 'van', 'Ginneken', ''],
  ['Rashindra', '', 'Manniesing', '']],
 'title': 'MemCNN: A Python/PyTorch package for creating memory-efficient invertible neural networks',
 'journal': '_Journal_of_Open_Source_Software_',
 'year': '2019',
 'date': '2019-07-30',
 'volume': '4',
 'number': '39',
 'month': '7',
 'pages': '1576',
 'issn': '2475-9066',
 'doi': 'https://doi.org/10.21105/joss.01576',
 'url': 'http://dx.doi.org/10.21105/joss.01576',
 'abstract': 'Neural networks are computational models that were originally inspired by biological neural networks like animal brains. These networks are composed of many small computational units called neurons that perform elementary calculations. Instead of explicitly programming the behavior of neural networks, these models can be trained to perform tasks, like classifying images, by presenting them examples. Sufficiently complex neural networks can automatical

In [ ]:
bib_

In [13]:
'sanc08b' in bib_author_keys['Clarisa-Sánchez']

True

In [40]:
s = b"S\'anchez"

In [45]:
s.decode('latex')

'I{\x0b{s}}gum'

In [36]:
'\\\'' in s

False

In [27]:
decode_name([s])

['Sánchez']

In [25]:
codecs.decode(s, 'ulatex')

"S'anchez"

In [36]:
latexcodec.codec.codecs.utf_32_decode(s)

TypeError: a bytes-like object is required, not 'str'

In [26]:
bib_items = {}
string_rules = {}
with open('/home/mart/Downloads/fullstrings.bib', 'rb') as f:
    content = f.read().decode('utf-8-sig')

for block in get_blocks(content):
    block_name, block_content = block
    if '@String' in block_name:
        k, v = [x.strip() for x in block_content.split('=')]
        string_rules[k] = v 

In [27]:
'_Journal_of_Open_Source_Software_' in string_rules

True

In [6]:
string_rules

{'_American_Journal_of_Transplantation_': '{American Journal of Transplantation}',
 '_Annals_of_the_American_Thoracic_Society_': '{}',
 '_Bulletin_of_the_Autralian_Mathematical_Society_': '{Bulletin of the Autralian Mathematical Society}',
 '_Computational_Pathology_and_Ophthalmic_Medical_Image_Analysis_': '{Computational Pathology and Ophthalmic Medical Image Analysis}',
 '_European_Congress_of_Pathology_': '{European Congress of Pathology}',
 '_European_Stroke_Journal_': '{European Stroke Journal}',
 '_Frontiers_in_ICT_': '{Frontiers in ICT}',
 '_GigaScience_': '{GigaScience}',
 '_Indagationes_Mathematicae_': '{Indagationes Mathematicae}',
 '_Infinite_Dimensional_Analysis_Quantum_Probability_and_Related_Topics_': '{Infinite Dimensional Analysis, Quantum Probability and Related Topics}',
 '_Interfase_Focus_': '{Interfase Focus}',
 '_International_Journal_of_Gynecological_Cancer_': '{International Journal of Gynaecological Cancer}',
 '_Journal_of_Invasive_Cardiology_': '{Journal of Inv

# 

In [12]:
boollist = []

for key, value  in string_rules.items():
    key = key.replace('_', ' ').strip()
    value = value.replace('}', '').replace('{', '')
    if key != value:
        print('key:', key)
        print('value:', value)
    boollist.append(key == value)
    
all(boollist)

key: Annals of the American Thoracic Society
value: 
key: Infinite Dimensional Analysis Quantum Probability and Related Topics
value: Infinite Dimensional Analysis, Quantum Probability and Related Topics
key: International Journal of Gynecological Cancer
value: International Journal of Gynaecological Cancer
key: Journal of the Belgian Society of Radiology
value: Journal of the Belgian Society of Nephrology
key: Nature Partner Journals Breast Cancer
value: NPJ Breast Cancer
key: NeuroImage Clinical
value: NeuroImage: Clinical
key: Public Health Action
value: 


False